<h1> 1.Časť - prieskumná analýza </h1>

V tejto analýze budeme pracovať s dvoma datasetmi personal kde budu osobné informácie a other, ktorý obshuje namerané hodnty pacientov z cukrovkou a bez nej táto informácia sa nachádza v stplci class.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


In [ ]:
dataP = pd.read_csv("personal_train.csv", index_col=0)
dataP.info()

V datasete personal máme 5 atributov ako môzeme vidieť po príkaze info. Z toho jeden čiselný a 4 kategorické.

In [ ]:
dataO = pd.read_csv("other_train.csv", index_col=0) #, na_values =["??"]
dataO.info()

V datasete other máme 21 atributov. Z toho 14 čiselných a 7 kategorických.
V tomto datasete sa už nachádzajú aj nulové hodnoty, keďže počet hodnôt v jednotlivých atribútoch sa líši. Spustením príkazu v nasledujúcej bunke zistíme koľko udajov obsahuje nulové hodnoty.
Tiež sme si všimli, že sa tu nachádza o 50 údajov viac v rovnakých atributoch ako v prvom datasete.

In [ ]:
dataO.shape[0] - dataO.dropna().shape[0]

In [ ]:
dataP.head() 

Z ukážky dát sme zistili, že niektoré hodnoty veku sú sabotáž ale keďže máme aj dátum tak budeme vedieť podozrivé hodnoty bez podozrivého dátumu doplniť. 

In [ ]:
dataO.head()

Po vypísaní ukážky dát sme si všimli, že stĺpce education-num a class sú kategorické aj keď sú číselné. Preto zmeníme ich typ z float64 na object.

In [ ]:
dataO = dataO.astype({'class':str,"education-num":str})

In [ ]:
dataP.describe()

In [ ]:
dataP.describe(exclude=np.number)

Describe nám ukázal len to čo už vieme, že z atribútu vek bude potrebne odstraniť prípadne neskôr doplniť podľa dátumu hodnoty, ktoré sú záporné a taktiež tie, ktoré sú väčšie ako sto keďže je nepravdepodobné, že bude niekto starší. Všetky ostatné hodnoty vyzerajú bzť v poriadku. Až na dátum, ktorý je niekedý zapísaný v inom formáte, čo bude tiež neskôr potrebné upraviť na jednotný formát.

In [ ]:
dataO.describe()

In [ ]:
dataO.describe(exclude=np.number)

In [ ]:
dataO.info()

In [ ]:
dataO.median()
#dat prec kategorické

In [ ]:
dataP.plot.box()

In [ ]:
dataO.plot.box()

In [ ]:
dataP.age.plot.hist(bins=30)

In [ ]:
dataP2 = dataP[(dataP['age']>0) & (dataP['age'] < 100)]

In [ ]:
dataP2.age.plot.hist(bins=30)

In [ ]:
#co vsetko histogramovat z tychto dat? treba to nejako upravovat??
dataO.std_oxygen.plot.hist(bins=30)

In [ ]:
#STATISTIKA - KATEGORICKE ATRIBUTY
#doplnanie dat? vysoke veky ale je datum narodenia doplnit z toho???????????
# pokial chceme spravit napr class - ma to problem s tým ze je to cislo, ako to obísť??
dataP.sex.value_counts().plot(kind = 'pie')

In [ ]:
sns.pairplot(dataO, hue = 'occupation')

In [ ]:
dataO.corr()